# Framework for predictions and portfolio forming

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time

# import the parquet library
import pyarrow.parquet as pq

# import model libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
# load 'basemodel.parquet'
#df = pd.read_parquet('basemodel.parquet')
df = pd.read_parquet('/kaggle/input/sign-prediction-datasets/basemodel.parquet')
prediction_cols = []
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y
0,10000,1986-02-28,-0.257143,11960.00000,0.0,0.0,0.0,1.0,0,0,0.0121,0.004058,0
1,10000,1986-03-31,0.365385,16330.00000,0.0,0.0,0.0,0.0,0,0,0.0766,0.000973,1
2,10000,1986-04-30,-0.098592,15172.00000,0.0,1.0,0.0,0.0,0,0,0.0548,0.001993,0
3,10000,1986-05-31,-0.222656,11793.87834,0.0,0.0,0.0,0.0,0,1,-0.0079,0.000163,0
4,10000,1986-06-30,-0.005025,11734.59375,0.0,0.0,0.0,0.0,0,2,0.0511,0.001569,0


In [3]:
# select the columns to be used for prediction
X_col = ['bull_D', 'bear_D', 'bull_W', 'bear_W', 'bull_M', 'bear_M', 'LMKT', 'IVOL']

In [4]:
# Convert 'date' to datetime format (if not already done) and sort the DataFrame
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a 'year' column based on the 'date' column
df['year'] = df['date'].dt.year

# Paper Replication - OLS and Logit, Expanding Window - No Hyperparameters
- They start with out of sample forecasting in 1932
- models will be named model_default

# My Experiments

## Machine Learning - Hyperparameter Tuning included in the process
- models to be named 'model_clas/reg_exp/roll'

### First expanding, then rolling
start predicting for 1932, expand the window until you reach X years, then roll it

##### 10 year window

In [5]:
# set the length of the rolling window
rolling_window = 10 # years

In [6]:
#################################
# RANDOM FOREST REGRESSION
#################################
start_time2 = time.time()  # Start timing

model_name = 'RF_reg_roll10'  # Name for storing Random Forest regression predictions

# Predefined set of values for hyperparameter tuning
HP1 = [10, 50, 100, 200]  # Possible values for n_estimators
HP2 = [None, 5, 10, 15]  # Possible values for max_depth

# Update the column name for storing Random Forest regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7

for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_HP2 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        for hp2 in HP2:
            model = RandomForestRegressor(n_estimators=hp1, max_depth=hp2, random_state=42, n_jobs = -1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_tune)  # Predict continuous values
            mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
            
            if mse < best_mse:  # Lower MSE is better
                best_mse = mse
                best_HP1 = hp1
                best_HP2 = hp2
    
    # Retrain on the entire training window (excluding tuning year) with the best hyperparameters
    model = RandomForestRegressor(n_estimators=best_HP1, max_depth=best_HP2, random_state=42, n_jobs = -1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best n_estimators: {best_HP1}, Best max_depth: {best_HP2}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")

end_time2 = time.time()
print(f"Total time: {end_time2 - start_time2:.2f} seconds")

Year 1933 - Best n_estimators: 200, Best max_depth: 10, Best MSE: 0.0911, Time: 21.74 seconds
Year 1934 - Best n_estimators: 10, Best max_depth: 10, Best MSE: 0.0913, Time: 22.73 seconds
Year 1935 - Best n_estimators: 200, Best max_depth: 10, Best MSE: 0.0992, Time: 29.71 seconds
Year 1936 - Best n_estimators: 50, Best max_depth: 10, Best MSE: 0.1101, Time: 31.51 seconds
Year 1937 - Best n_estimators: 50, Best max_depth: 10, Best MSE: 0.0973, Time: 34.71 seconds
Year 1938 - Best n_estimators: 10, Best max_depth: 10, Best MSE: 0.0785, Time: 34.17 seconds
Year 1939 - Best n_estimators: 100, Best max_depth: 10, Best MSE: 0.0876, Time: 37.56 seconds
Year 1940 - Best n_estimators: 200, Best max_depth: 5, Best MSE: 0.0866, Time: 38.62 seconds
Year 1941 - Best n_estimators: 10, Best max_depth: 10, Best MSE: 0.1129, Time: 36.89 seconds
Year 1942 - Best n_estimators: 100, Best max_depth: 10, Best MSE: 0.1216, Time: 40.08 seconds
Year 1943 - Best n_estimators: 200, Best max_depth: 5, Best MSE: 0

##### 20 year window

In [7]:
rolling_window = 20 # years

## Forming Portfolios, Value-weighted portfolio returns

In [8]:
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,year,RF_reg_roll10
0,11148,1926-08-31,0.109924,76287.5,1.0,0.0,4.0,0.0,0,0,0.0318,0.000040,1,1926,NaN
1,10874,1926-08-31,0.168142,16500.0,0.0,0.0,2.0,0.0,2,0,0.0318,0.000537,1,1926,NaN
2,12968,1926-08-31,-0.006464,55683.0,0.0,0.0,0.0,0.0,0,0,0.0318,0.000007,0,1926,NaN
3,12976,1926-08-31,0.156041,41650.0,1.0,0.0,0.0,0.0,0,0,0.0318,0.000070,1,1926,NaN
4,12984,1926-08-31,0.046875,11323.0,1.0,0.0,0.0,0.0,0,0,0.0318,0.000874,1,1926,NaN


In [9]:
prediction_cols
# prediction_cols = ['logit_default','OLS_default','logit_roll6','DT_reg_roll']

['RF_reg_roll10']

In [10]:
portfolio = df[['date', 'RET', 'ME', 'y'] + prediction_cols].copy()
portfolio['date'] = pd.to_datetime(portfolio['date'])

# drop rows with missing values
portfolio.dropna(inplace=True)

portfolio.head()

,date,RET,ME,y,RF_reg_roll10
49083,1933-01-31,0.036765,31918.875,1,1.000000
49084,1933-01-31,0.121849,7142.250,1,0.309865
49085,1933-01-31,0.015625,1560.000,1,1.000000
49086,1933-01-31,-0.084337,4275.000,0,0.000000
49087,1933-01-31,-0.037500,447581.750,0,0.178455


In [11]:
portfolio.tail()

,date,RET,ME,y,RF_reg_roll10
3627748,2022-07-31,-0.106939,3.731571e+06,0,0.432648
3627749,2022-07-31,0.020969,5.066263e+03,1,0.023251
3627750,2022-07-31,0.184785,1.280287e+06,1,0.994785
3627751,2022-07-31,0.108881,3.475711e+04,1,0.072864
3627752,2022-07-31,0.323765,9.311106e+08,1,0.629270


In [12]:
# Initialize an empty DataFrame to store value-weighted returns for each model
vwreturns = pd.DataFrame(portfolio['date'].unique(), columns=['date'])  # Ensures all dates are included

for pred_col in prediction_cols:
    # Calculate deciles for this prediction
    decile_col = f'decile_{pred_col}'
    portfolio[decile_col] = portfolio.groupby(['date'])[pred_col].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))
    
    # Determine position based on deciles
    position_col = f'position_{pred_col}'
    portfolio[position_col] = np.where(portfolio[decile_col] == 9, 1, np.where(portfolio[decile_col] == 0, -1, 0))
    
    # Calculate the value-weighted return for this prediction
    vwret_col = f'vwreturn_{pred_col}'
    vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
    
    # Merge the temporary value-weighted returns with the main vwreturns DataFrame
    vwreturns = vwreturns.merge(vwreturns_temp, on='date', how='left')

# Ensure the 'date' column is the first column and is sorted
vwreturns = vwreturns.sort_values('date').reset_index(drop=True)


/tmp/ipykernel_26/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)


In [13]:
vwreturns.head()

,date,vwreturn_RF_reg_roll10
0,1933-01-31,0.006215
1,1933-02-28,0.065318
2,1933-03-31,0.026626
3,1933-04-30,-0.003213
4,1933-05-31,-0.003773


### Compare to market data

In [14]:
#market = pd.read_csv('FF3_clean.csv')
market = pd.read_csv('/kaggle/input/sign-prediction-datasets/FF3_clean.csv')

In [15]:
market.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-31,2.96,-2.56,-2.43,0.22
1,1926-08-31,2.64,-1.17,3.82,0.25
2,1926-09-30,0.36,-1.40,0.13,0.23
3,1926-10-31,-3.24,-0.09,0.70,0.32
4,1926-11-30,2.53,-0.10,-0.51,0.31


In [16]:
# create a new 'Mkt' which is a sum of Mkt-RF and RF
market['Mkt'] = market['Mkt-RF'] + market['RF']

# divide all columns by 100 except 'date'
market.iloc[:, 1:] = market.iloc[:, 1:] / 100

#set the 'date' column to datetime format
market['date'] = pd.to_datetime(market['date'])

# merge the market data (only date and Mkt columns) with the vwreturns DataFrame
vwreturns = vwreturns.merge(market[['date', 'Mkt']], on='date', how='left')

# transform all columns (except 'date') to a log: log(x+1) and save the result as lvwreturns
lvwreturns = vwreturns.copy()
lvwreturns.iloc[:, 1:] = np.log(vwreturns.iloc[:, 1:] + 1)

In [17]:
vwreturns.head()

,date,vwreturn_RF_reg_roll10,Mkt
0,1933-01-31,0.006215,0.0126
1,1933-02-28,0.065318,-0.1527
2,1933-03-31,0.026626,0.0333
3,1933-04-30,-0.003213,0.3895
4,1933-05-31,-0.003773,0.2147


In [18]:
lvwreturns.head()

,date,vwreturn_RF_reg_roll10,Mkt
0,1933-01-31,0.006196,0.012521
1,1933-02-28,0.063273,-0.165700
2,1933-03-31,0.026278,0.032758
3,1933-04-30,-0.003218,0.328944
4,1933-05-31,-0.003780,0.194497


In [19]:
lvwreturns.describe()

,date,vwreturn_RF_reg_roll10,Mkt
count,1074,1074.000000,1074.000000
mean,1977-11-05 12:12:04.022346368,0.012620,0.008968
min,1933-01-31 00:00:00,-0.073611,-0.272203
25%,1955-06-07 12:00:00,0.003123,-0.017248
50%,1977-11-15 00:00:00,0.007217,0.013064
75%,2000-03-23 06:00:00,0.016244,0.038162
max,2022-07-31 00:00:00,0.193999,0.328944
std,NaN,0.018138,0.048186


In [20]:
## plot histograms of the value-weighted returns for each model and the market in lvwreturns
#plt.figure(figsize=(12, round(len(prediction_cols)/2) * 5 ))
#
#for i, pred_col in enumerate(prediction_cols):
#    plt.subplot(len(prediction_cols)/2 +1, 2, i+1)
#    plt.hist(lvwreturns[f'vwreturn_{pred_col}'], bins=50, color='skyblue', edgecolor='black')
#    plt.title(f'Value-Weighted Return - {pred_col}')
#    plt.xlabel('Value-Weighted Return')
#    plt.ylabel('Frequency')
#    # calculate mean, skewness and kurtosis and add their values to the plot as a text, aligning to the top right corner
#    mean = lvwreturns[f'vwreturn_{pred_col}'].mean()
#    skewness = lvwreturns[f'vwreturn_{pred_col}'].skew()
#    kurtosis = lvwreturns[f'vwreturn_{pred_col}'].kurtosis()
#
#    plt.text(0.95, 0.95, f'Mean: {mean:.4f}\nSkewness: {skewness:.4f}\nKurtosis: {kurtosis:.4f}', ha='right', va='top', transform=plt.gca().transAxes)
#
#
#
#plt.subplot(round(len(prediction_cols)/2) +1, 2, len(prediction_cols)+1)
#plt.title('Value-Weighted Return - Market')
#plt.xlabel('Value-Weighted Return')
#plt.ylabel('Frequency')
#plt.hist(lvwreturns['Mkt'], bins=50, color='skyblue', edgecolor='black')
#mean = lvwreturns['Mkt'].mean()
#skewness = lvwreturns['Mkt'].skew()
#kurtosis = lvwreturns['Mkt'].kurtosis()
#plt.text(0.95, 0.95, f'Mean: {mean:.4f}\nSkewness: {skewness:.4f}\nKurtosis: {kurtosis:.4f}', ha='right', va='top', transform=plt.gca().transAxes)
#
#plt.tight_layout()
#plt.show()
#

In [21]:
## plot cumulative sums of the value-weighted log returns
#plt.figure(figsize=(12, 6))
#plt.plot(lvwreturns['date'], lvwreturns.iloc[:, 1:].cumsum())
#plt.title('Cumulative Value-Weighted Log Returns')
#plt.xlabel('Date')
#plt.ylabel('Cumulative Value-Weighted Log Returns')
#plt.legend(prediction_cols + ['Market'])
#plt.show()


In [22]:
# save the lvwreturns and portfolio DataFrame to a parquet file into 'outputs' folder

# for reproducibility and visualization purposes
lvwreturns.to_parquet('base_lvwreturns_reg3.parquet')
portfolio.to_parquet('base_portfolio_reg3.parquet')

# save vwreturns DataFrame to a .dta file into 'outputs' folder
#vwreturns.to_stata('outputs/vwreturns.dta') # for backtasting in R - we need normal returns, not log returns
